# Demo 1: Teljes Data Engineering pipeline Python-ban

**BME Adatmérnöki alapok – 1. hét**

Ez a demó bemutatja a teljes Data Engineering életciklust (Generálás → Bevitel → Tárolás → Transzformáció → Kiszolgálás) kizárólag Python használatával, Google Colab-on futtathatóan, Docker nélkül.

**Tartomány:** Időjárás/IoT – magyar városok hőmérséklet-elemzése

```
Generálás → Bevitel (Ingestion) → Tárolás → Transzformáció → Kiszolgálás (Serving)
                                                    ↕
                                              Adatminőség (Undercurrents)
```

In [ ]:
# Csomagok telepítése (Colab-on futtatva)
!pip install -q faker requests pandas matplotlib seaborn

import faker
import requests
import pandas as pd
import sqlite3
import json
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Magyar locale és stílus
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100
sns.set_style('whitegrid')
print("Csomagok betöltve!")

## 1. Generálás (Generation)

A Faker könyvtár segítségével 10 időjárás-állomás metaadatait generáljuk magyar városokhoz.
Ez reprezentálja a „forrásrendszert” (source system), amely az adatokat előállítja.

In [ ]:
# === 1. GENERÁLÁS: Időjárás-állomások metaadatai ===
from faker import Faker
fake = Faker('hu_HU')

# Magyar városok valós koordinátákkal
cities = [
    {"city": "Budapest",  "lat": 47.4979, "lon": 19.0402, "elevation": 96},
    {"city": "Debrecen",  "lat": 47.5316, "lon": 21.6273, "elevation": 121},
    {"city": "Szeged",    "lat": 46.2530, "lon": 20.1414, "elevation": 75},
    {"city": "Miskolc",   "lat": 48.1035, "lon": 20.7784, "elevation": 130},
    {"city": "Pécs",      "lat": 46.0727, "lon": 18.2323, "elevation": 153},
    {"city": "Győr",      "lat": 47.6875, "lon": 17.6504, "elevation": 108},
    {"city": "Nyíregyháza","lat": 47.9554, "lon": 21.7167, "elevation": 118},
    {"city": "Kecskemét", "lat": 46.9063, "lon": 19.6897, "elevation": 120},
    {"city": "Székesfehérvár","lat": 47.1860, "lon": 18.4221, "elevation": 110},
    {"city": "Sopron",    "lat": 47.6817, "lon": 16.5845, "elevation": 214},
]

stations_df = pd.DataFrame(cities)
stations_df['station_id'] = [f"HU-{i+1:03d}" for i in range(len(cities))]
stations_df['installed'] = [fake.date_between(start_date='-5y', end_date='-1y') for _ in range(len(cities))]

print(f"Generált állomások száma: {len(stations_df)}")
stations_df

## 2. Bevitel – Ingestion (Pull minta: REST API)

Az Open-Meteo API segítségével (ingyenes, nem igényel API kulcsot) 30 nap óránkénti időjárás-adatait kérjük le minden állomáshoz.
Ez egy **Pull ingestion** minta – aktívan lekérjük az adatokat a forrásból REST API-n keresztül.

> **API info:** [Open-Meteo](https://open-meteo.com/) – Nyilvános, ingyenes meteorológiai API. Nincs szükség regisztrációra vagy API kulcsra.

In [ ]:
# === 2. INGESTION: Open-Meteo API – 30 nap óránkénti adatok ===
BASE_URL = "https://api.open-meteo.com/v1/forecast"

end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")

all_weather = []
for _, station in stations_df.iterrows():
    params = {
        "latitude": station["lat"],
        "longitude": station["lon"],
        "hourly": "temperature_2m,relative_humidity_2m,wind_speed_10m,precipitation",
        "start_date": start_date,
        "end_date": end_date,
        "timezone": "Europe/Budapest"
    }

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        hourly = data["hourly"]

        for i in range(len(hourly["time"])):
            all_weather.append({
                "station_id": station["station_id"],
                "timestamp": hourly["time"][i],
                "temperature_c": hourly["temperature_2m"][i],
                "humidity_pct": hourly["relative_humidity_2m"][i],
                "wind_speed_kmh": hourly["wind_speed_10m"][i],
                "precipitation_mm": hourly["precipitation"][i],
            })
        print(f"  {station['city']:20s} – {len(hourly['time']):5d} mérés betöltve")
    else:
        print(f"  {station['city']:20s} – HIBA: {response.status_code}")

weather_df = pd.DataFrame(all_weather)
weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"])
print(f"\nÖsszesen {len(weather_df):,} mérési adat betöltve ({len(stations_df)} állomás × ~{len(weather_df)//len(stations_df)} óra)")
weather_df.head()

## 3. Tárolás (Storage) – SQLite adatbázis

Az adatokat SQLite adatbázisban tároljuk – ez egy könnyűsúlyú relációs adatbázis.
Éles környezetben ez PostgreSQL, data lake vagy más megoldás lehetne.
Létrehozunk egy „nyers zónát” (raw zone), ahová az eredeti adatokat mentjük.

In [ ]:
# === 3. STORAGE: SQLite adatbázis (raw zóna) ===
DB_PATH = "weather_pipeline.db"

# Töröljük a korábbi DB-t ha létezik
if os.path.exists(DB_PATH):
    os.remove(DB_PATH)

conn = sqlite3.connect(DB_PATH)

# Táblák létrehozása
conn.execute("""
    CREATE TABLE stations (
        station_id TEXT PRIMARY KEY,
        city TEXT NOT NULL,
        latitude REAL,
        longitude REAL,
        elevation INTEGER,
        installed DATE
    )
""")

conn.execute("""
    CREATE TABLE raw_weather (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        station_id TEXT REFERENCES stations(station_id),
        timestamp DATETIME NOT NULL,
        temperature_c REAL,
        humidity_pct REAL,
        wind_speed_kmh REAL,
        precipitation_mm REAL
    )
""")

# Adatok betöltése
stations_df.to_sql("stations", conn, if_exists="replace", index=False)
weather_df.to_sql("raw_weather", conn, if_exists="replace", index=False)
conn.commit()

# Ellenőrzés
station_count = pd.read_sql("SELECT COUNT(*) as cnt FROM stations", conn).iloc[0, 0]
weather_count = pd.read_sql("SELECT COUNT(*) as cnt FROM raw_weather", conn).iloc[0, 0]
print(f"SQLite adatbázis létrehozva: {DB_PATH}")
print(f"  stations tábla:    {station_count:>8,} sor")
print(f"  raw_weather tábla: {weather_count:>8,} sor")
print(f"  DB méret:          {os.path.getsize(DB_PATH) / 1024:.1f} KB")

## 4. Transzformáció (Transformation)

Itt tisztítjuk, gazdagítjuk és aggregáljuk a nyers adatokat. Lépések:
1. **Adattisztítás** – null értékek kezelése, outlier szűrés
2. **JOIN** – állomás metaadatokkal való összekapcsolás
3. **Napi aggregáció** – óránkénti adatokból napi statisztikák
4. **Feature engineering** – mozgóátlag, hőingás
5. **Anomália detekció** – z-score alapú kilógó értékek azonosítása

In [ ]:
# === 4. TRANSZFORMÁCIÓ ===

# 4.1 – Betöltés SQLite-ból
raw = pd.read_sql("""
    SELECT w.*, s.city, s.elevation
    FROM raw_weather w
    JOIN stations s ON w.station_id = s.station_id
""", conn)
raw["timestamp"] = pd.to_datetime(raw["timestamp"])

print(f"Betöltött sorok: {len(raw):,}")
print(f"Null értékek:\n{raw.isnull().sum()}")

# 4.2 – Tisztítás: null kezelés, outlier szűrés
raw = raw.dropna(subset=["temperature_c"])
raw = raw[(raw["temperature_c"] > -40) & (raw["temperature_c"] < 50)]
raw = raw[(raw["humidity_pct"] >= 0) & (raw["humidity_pct"] <= 100)]
print(f"\nTisztítás után: {len(raw):,} sor")

# 4.3 – Napi aggregáció
raw["date"] = raw["timestamp"].dt.date
daily = raw.groupby(["station_id", "city", "date"]).agg(
    avg_temp=("temperature_c", "mean"),
    min_temp=("temperature_c", "min"),
    max_temp=("temperature_c", "max"),
    avg_humidity=("humidity_pct", "mean"),
    avg_wind=("wind_speed_kmh", "mean"),
    total_precip=("precipitation_mm", "sum"),
    measurement_count=("temperature_c", "count")
).reset_index()
daily["date"] = pd.to_datetime(daily["date"])

# 4.4 – Feature engineering: mozgóátlag, napi hőingás
daily["temp_7d_mavg"] = daily.groupby("city")["avg_temp"].transform(
    lambda x: x.rolling(7, min_periods=1).mean()
)
daily["temp_range"] = daily["max_temp"] - daily["min_temp"]

# 4.5 – Anomália detekció (z-score alapú)
city_stats = daily.groupby("city")["avg_temp"].agg(["mean", "std"]).reset_index()
city_stats.columns = ["city", "city_mean", "city_std"]
daily = daily.merge(city_stats, on="city")
daily["z_score"] = (daily["avg_temp"] - daily["city_mean"]) / daily["city_std"]
daily["is_anomaly"] = daily["z_score"].abs() > 2.0

print(f"\nNapi aggregáció: {len(daily):,} sor")
print(f"Anomáliák száma: {daily['is_anomaly'].sum()}")
print(f"\nVárosi statisztikák:")
print(daily.groupby("city")[["avg_temp", "total_precip"]].mean().round(1).to_string())

## 5. Kiszolgálás (Serving) – Vizualizáció

Az utolsó fázis – az adatok kiszolgálása a végfelhasználóknak.
Éles környezetben ez lehetne dashboard (Tableau, PowerBI), API végpont, vagy ML modell bemenet.
Mi egy 4 paneles analitikai dashboard-ot készítünk.

In [ ]:
# === 5. SERVING: Analitikai dashboard ===
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle("Időjárás-elemzés – Magyar városok (utolsó 30 nap)", fontsize=16, fontweight='bold')

# --- 5.1 Hőmérséklet trend (top 5 város) ---
ax1 = axes[0, 0]
top_cities = ["Budapest", "Debrecen", "Szeged", "Pécs", "Sopron"]
for city in top_cities:
    city_data = daily[daily["city"] == city].sort_values("date")
    ax1.plot(city_data["date"], city_data["temp_7d_mavg"], label=city, linewidth=2)
ax1.set_title("Hőmérséklet trend (7 napos mozgóátlag)")
ax1.set_ylabel("Hőmérséklet (°C)")
ax1.legend(loc="upper left", fontsize=8)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m.%d'))
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# --- 5.2 Átlaghőmérséklet városonként (barplot) ---
ax2 = axes[0, 1]
city_avg = daily.groupby("city")["avg_temp"].mean().sort_values(ascending=True)
colors = sns.color_palette("coolwarm", len(city_avg))
bars = ax2.barh(city_avg.index, city_avg.values, color=colors)
ax2.set_title("Átlagos hőmérséklet városonként")
ax2.set_xlabel("Hőmérséklet (°C)")
for bar, val in zip(bars, city_avg.values):
    ax2.text(val + 0.1, bar.get_y() + bar.get_height()/2, f"{val:.1f}°C", va='center', fontsize=8)

# --- 5.3 Hőmérséklet heatmap ---
ax3 = axes[1, 0]
pivot = daily.pivot_table(index="city", columns="date", values="avg_temp")
# Dátum címkék egyszerűsítése
pivot.columns = [d.strftime("%m.%d") for d in pivot.columns]
# Csak minden N-edik címke megjelenítése
n_cols = len(pivot.columns)
show_every = max(1, n_cols // 10)
sns.heatmap(pivot, ax=ax3, cmap="RdYlBu_r", cbar_kws={"label": "°C"},
            xticklabels=show_every)
ax3.set_title("Hőmérséklet heatmap (város × nap)")
ax3.set_xlabel("")
ax3.tick_params(axis='x', rotation=45)

# --- 5.4 Csapadék összehasonlítás ---
ax4 = axes[1, 1]
precip_total = daily.groupby("city")["total_precip"].sum().sort_values(ascending=False)
ax4.bar(range(len(precip_total)), precip_total.values, color=sns.color_palette("Blues_r", len(precip_total)))
ax4.set_xticks(range(len(precip_total)))
ax4.set_xticklabels(precip_total.index, rotation=45, ha="right")
ax4.set_title("Összes csapadék (30 nap)")
ax4.set_ylabel("Csapadék (mm)")

plt.tight_layout()
plt.savefig("weather_dashboard.png", dpi=150, bbox_inches="tight")
plt.show()
print("Dashboard mentve: weather_dashboard.png")

## 6. Adatminőség (Undercurrents)

Az adatminőség az egyik „undercurrent”, amely az életciklus minden fázisát áthatja.
Alapvető minőségi ellenőrzéseket végzünk: teljeség, frissesség, tartomány-validáció.

In [ ]:
# === 6. ADATMINŐSÉG (Undercurrents) ===
print("=" * 60)
print("ADATMINŐSÉGI JELENTÉS")
print("=" * 60)

checks = []

# 6.1 – Teljesség (Completeness)
total_expected = len(stations_df) * 31 * 24  # 10 állomás × 31 nap × 24 óra
total_actual = len(weather_df)
completeness = total_actual / total_expected * 100
passed = completeness > 95
checks.append(("Teljesség (completeness)", f"{completeness:.1f}%", "PASS" if passed else "FAIL"))
print(f"\n1. Teljesség: {total_actual:,} / {total_expected:,} = {completeness:.1f}% {chr(10003) if passed else chr(10007)}")

# 6.2 – Frissesség (Freshness)
latest = weather_df["timestamp"].max()
freshness_hours = (datetime.now() - latest).total_seconds() / 3600
passed = freshness_hours < 48
checks.append(("Frissesség (freshness)", f"{freshness_hours:.0f} óra", "PASS" if passed else "FAIL"))
print(f"2. Frissesség: legutóbbi adat {freshness_hours:.0f} órája {chr(10003) if passed else chr(10007)}")

# 6.3 – Tartomány (Range validation)
temp_min, temp_max = weather_df["temperature_c"].min(), weather_df["temperature_c"].max()
passed = temp_min > -50 and temp_max < 60
checks.append(("Hőmérséklet tartomány", f"[{temp_min:.1f}, {temp_max:.1f}]°C", "PASS" if passed else "FAIL"))
print(f"3. Hőmérséklet tartomány: [{temp_min:.1f}, {temp_max:.1f}]°C {chr(10003) if passed else chr(10007)}")

# 6.4 – Null arány
null_pct = weather_df.isnull().mean() * 100
max_null = null_pct.max()
passed = max_null < 5
checks.append(("Null arány (max)", f"{max_null:.2f}%", "PASS" if passed else "FAIL"))
print(f"4. Maximális null arány: {max_null:.2f}% {chr(10003) if passed else chr(10007)}")

# 6.5 – Egyediség (állomások)
unique_stations = weather_df["station_id"].nunique()
passed = unique_stations == len(stations_df)
checks.append(("Egyedi állomások", f"{unique_stations}/{len(stations_df)}", "PASS" if passed else "FAIL"))
print(f"5. Egyedi állomások: {unique_stations}/{len(stations_df)} {chr(10003) if passed else chr(10007)}")

# Összesítés
print("\n" + "=" * 60)
pass_count = sum(1 for c in checks if c[2] == "PASS")
print(f"Eredmény: {pass_count}/{len(checks)} ellenőrzés SIKERES")
print("=" * 60)

# Cleanup
conn.close()
if os.path.exists(DB_PATH):
    os.remove(DB_PATH)

## Összefoglalás

| DE Lifecycle fázis | Megvalósítás | Python eszköz |
|---|---|---|
| Generálás | Állomás metaadatok | Faker |
| Bevitel (Ingestion) | REST API lekérés | requests (Pull minta) |
| Tárolás (Storage) | SQLite adatbázis | sqlite3 |
| Transzformáció | Tisztítás, aggregáció, feature eng. | pandas |
| Kiszolgálás (Serving) | Dashboard vizualizáció | matplotlib, seaborn |
| Adatminőség | Minőségi ellenőrzések | assertions |

---

Ez a teljes pipeline **pure Python-ban** futott, Docker és felhőszolgáltatás nélkül.
Éles környezetben ezeket az eszközöket modern, skálázható megoldásokkal helyettesítjük – ezt mutatja a 2. demó.